In [26]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns


In [27]:
df = pd.read_csv('circles.csv')  

In [28]:
#Prelimnary data exploration
print(df.head())
print(df.info())
print(df.describe())
df.isnull().sum()  
df[df.duplicated(keep=False)]

           x          y  class
0    3.15676  116.12252      6
1   16.14436   16.81660     11
2  100.31212   64.99025     53
3   -1.33773   84.81772      4
4  104.37328   62.42373     53
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       10000 non-null  float64
 1   y       10000 non-null  float64
 2   class   10000 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 234.5 KB
None
                  x             y         class
count  10000.000000  10000.000000  10000.000000
mean      89.981670     90.003827     49.500000
std       57.457467     57.559719     28.867513
min       -5.000000     -4.999860      0.000000
25%       40.237885     40.133840     24.750000
50%       89.999995     90.000080     49.500000
75%      139.898800    139.977695     74.250000
max      184.999920    185.000000     99.000000


,x,y,class


In [29]:
#Preprocessing 

for col in ['x','y']:
    mean = df[col].mean()
    std = df[col].std()
    df[col] = (df[col] - mean) / std

In [30]:
df.head()

,x,y,class
0,-1.511116,0.453767,6
1,-1.285078,-1.271501,11
2,0.179793,-0.434567,53
3,-1.589339,-0.090100,4
4,0.250474,-0.479156,53


In [ ]:
#Kmeans Clustering algorithm 
def euclidean_distance(point, data):
    return np.sqrt(np.sum((point - data) ** 2))

class KMeans: 
    #initialize k and max iterations (to prevent infinite loops)
    def __init__(self, k, max_iterations=100):
        self.k = k
        self.max_iterations = max_iterations
    
    #Main Clustering function
    def fit(self,data):
        
        #initalize k random centroids 
        min_c, max_c = np.min(data, axis=0), np.max(data, axis=0)
        self.centroids = [np.uniform(min_c, max_c) for _ in range(self.k)]
            
        # Iterate to refine centroids till convergence or max iterations
        curr_iter = 0
        previous_centroids = None
        while previous_centroids is None or np.not_equal(previous_centroids, self.centroids).any() and curr_iter < self.max_iterations:
            
            #sort points into clusters by computing distances to centroids and choosing smallest distance
            sorted_pts = [[] for _ in range(self.k)]
            for point in data:
                dists  = euclidean_distance(point, self.centroids)
                centroid_idx = np.argmin(dists)
                sorted_pts[centroid_idx].append(point)
            
        